# Q1. You are working on a machine learning project where you have a dataset containing numerical and categorical features. You have identified that some of the features are highly correlated and there are missing values in some of the columns. You want to build a pipeline that automates the feature engineering process and handles the missing values.

#### Design a pipeline that includes the following steps:

* Use an automated feature selection method to identify the important features in the dataset.

Create a numerical pipeline that includes the following steps:
* Impute the missing values in the numerical columns using the mean of the column values.
* Scale the numerical columns using standardisation.

Create a categorical pipeline that includes the following steps:
* Impute the missing values in the categorical columns using the most frequent value of the columns.
* One-hot encode the categorical columns.
* Combine the numerical and categorical pipelines using a ColumnTransformer.
* Use a Random Forest Classifier to build the final model.
* Evaluate the accuracy of the model on the test dataset.


Note! Your solution should include code snippets for each step of the pipeline, and a brief explanation of each step. You should also provide an interpretation of the results and suggest possible improvements for the pipeline.

In [22]:
import seaborn as sns
df=sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [26]:
df['time'].unique()

array([0, 1])

In [27]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df['time']=encoder.fit_transform(df['time'])

In [28]:
df['time'].unique()

array([0, 1])

In [29]:
## independent and dependent features
X=df.drop(labels=['time'],axis=1)
y=df['time']

In [30]:
X.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [31]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.33,random_state=42)

In [32]:
X_train.head()

,total_bill,tip,sex,smoker,day,size
114,25.71,4.00,Female,No,Sun,3
65,20.08,3.15,Male,No,Sat,3
132,11.17,1.50,Female,No,Thur,2
207,38.73,3.00,Male,Yes,Sat,4
162,16.21,2.00,Female,No,Sun,3


In [34]:
from sklearn.pipeline import Pipeline

from sklearn.impute import SimpleImputer ## Handling Missing Values
from sklearn.preprocessing import StandardScaler## Feature scaling

from sklearn.preprocessing import OneHotEncoder## handling Categorical features


from sklearn.compose import ColumnTransformer

In [33]:
numerical_cols = ['total_bill', 'tip','size']
categorical_cols = ['sex', 'smoker','day']

In [35]:
## Feature Engineering Automation
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='mean')), ##missing values
        ('scaler',StandardScaler())## feature scaling 
    ]

)

#categorical Pipeline
cat_pipeline=Pipeline(
                steps=[
                ('imputer',SimpleImputer(strategy='most_frequent')), ## handling Missing values
                ('onehotencoder',OneHotEncoder()) ## Categorical features to numerical
                ]

            )  

In [36]:
# Combining the numerical and categorical pipelines using a ColumnTransformer.

preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)

])

In [37]:
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

In [38]:
from sklearn.ensemble import RandomForestClassifier

In [39]:
rfc = RandomForestClassifier(n_estimators=100 , max_depth=10)

In [40]:
rfc.fit(X_train, y_train)

RandomForestClassifier(max_depth=10)

In [41]:
y_pred=rfc.predict(X_test)

In [43]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [44]:
print(confusion_matrix(y_pred,y_test))
print(accuracy_score(y_pred,y_test))
print(classification_report(y_pred,y_test))

[[59  1]
 [ 1 20]]
0.9753086419753086
              precision    recall  f1-score   support

           0       0.98      0.98      0.98        60
           1       0.95      0.95      0.95        21

    accuracy                           0.98        81
   macro avg       0.97      0.97      0.97        81
weighted avg       0.98      0.98      0.98        81



### The overall accuracy of the model is 0.975, which means that it correctly predicted 97.5% of the instances in the test set.

### Improvements for the pipeline.

One possible improvement for the pipeline could be to use a more sophisticated feature engineering technique to extract more informative features from the dataset. This could lead to a more accurate model. Another improvement could be to use a more advanced machine learning algorithm, such as a neural network or a gradient boosting machine, to further improve the accuracy of the predictions. Additionally, collecting more data and balancing the classes could also improve the model's performance.

# Q2. Build a pipeline that includes a random forest classifier and a logistic regression classifier, and then use a voting classifier to combine their predictions. Train the pipeline on the iris dataset and evaluate its accuracy.

In [94]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import VotingClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), [0, 1, 2, 3]),
        ('cat', OneHotEncoder(), [])
    ])

# Define the pipeline
rfc = RandomForestClassifier(random_state=42)
lr = LogisticRegression(random_state=42)
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('voting_classifier', VotingClassifier(estimators=[('rfc', rfc), ('lr', lr)], voting='soft'))])

# Fit the pipeline
pipeline.fit(X_train, y_train)

# Evaluate the pipeline
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 1.0
